In [ ]:
# Install necessary libraries
!pip install --upgrade google-cloud-language
!pip install pandas

import pandas as pd
from google.cloud import language_v1
from google.colab import drive
import os

# Authenticate with Google Cloud (ensure you have set up your Google Cloud project and enabled the Natural Language API)
# Upload your service account key file to authenticate
from google.colab import files
uploaded = files.upload()
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = list(uploaded.keys())[0]



Saving driven-crane-444705-b1-6f0898087820.json to driven-crane-444705-b1-6f0898087820 (2).json
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
drive.mount('/content/gdrive')
filepath = '/content/gdrive/MyDrive/EAI6010/Mod3/Tweets.csv'

# Initialize Google Cloud Natural Language client
client = language_v1.LanguageServiceClient()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Function to preprocess text
def preprocess_text(text):
    """Clean and prepare text for analysis."""
    import re
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters
    return text

# Function to analyze sentiment using Google Cloud NLP
def analyze_sentiment(text):
    """Analyze sentiment using Google Cloud NLP API."""
    document = language_v1.Document(content=text, type_=language_v1.Document.Type.PLAIN_TEXT)
    response = client.analyze_sentiment(document=document)
    sentiment = response.document_sentiment
    return sentiment.score, sentiment.magnitude

# Load dataset and preview data
data = pd.read_csv(filepath, encoding='latin1')
print(data.columns)

Index(['tweet_id', 'airline_sentiment', 'airline_sentiment_confidence',
       'negativereason', 'negativereason_confidence', 'airline',
       'airline_sentiment_gold', 'name', 'negativereason_gold',
       'retweet_count', 'text', 'tweet_coord', 'tweet_created',
       'tweet_location', 'user_timezone'],
      dtype='object')


In [ ]:
import time
# Select relevant columns
data = data[['text', 'airline_sentiment']]  # Columns for text and sentiment
data['text'] = data['text'].apply(preprocess_text)

# Process data in batches
batch_size = 10
results = []

for start_idx in range(0, len(data), batch_size):
    batch = data.iloc[start_idx:start_idx + batch_size]
    print(f"Processing batch {start_idx} to {start_idx + len(batch) - 1}...")

    batch_results = []
    for i, row in batch.iterrows():
        try:
            text = row['text']
            label = row['airline_sentiment']
            score, magnitude = analyze_sentiment(text)
            batch_results.append({'text': text, 'true_label': label, 'score': score, 'magnitude': magnitude})
        except Exception as e:
            print(f"Error processing row {i}: {e}")
            continue

    # Append batch results to main results
    results.extend(batch_results)

    # Optional: Add a delay to avoid hitting API rate limits
    time.sleep(1)  # Adjust as needed

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Evaluate performance
from sklearn.metrics import classification_report

# Map Google NLP score to sentiment labels
def map_sentiment(score):
    if score > 0.25:
        return "positive"
    elif score < -0.25:
        return "negative"
    else:
        return "neutral"

results_df['predicted_label'] = results_df['score'].apply(map_sentiment)
print(classification_report(results_df['true_label'], results_df['predicted_label']))

Processing batch 0 to 9...
Processing batch 10 to 19...
Processing batch 20 to 29...
Processing batch 30 to 39...
Processing batch 40 to 49...
Processing batch 50 to 59...
Processing batch 60 to 69...
Processing batch 70 to 79...
Processing batch 80 to 89...
Processing batch 90 to 99...
Processing batch 100 to 109...
Processing batch 110 to 119...
Processing batch 120 to 129...
Processing batch 130 to 139...
Processing batch 140 to 149...
Processing batch 150 to 159...
Processing batch 160 to 169...
Processing batch 170 to 179...
Processing batch 180 to 189...
Processing batch 190 to 199...
Processing batch 200 to 209...
Processing batch 210 to 219...
Processing batch 220 to 229...
Processing batch 230 to 239...
Processing batch 240 to 249...
Processing batch 250 to 259...
Processing batch 260 to 269...
Processing batch 270 to 279...
Processing batch 280 to 289...
Processing batch 290 to 299...
Processing batch 300 to 309...
Processing batch 310 to 319...
Processing batch 320 to 329...